In [1]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(project_root)

from examples.mia.LOS.utils.data_processing import get_mimic_dataset, get_mimic_dataloaders
from examples.mia.LOS.utils.model import MimicLR, create_trained_model_and_metadata




In [2]:

# Generate the dataset and dataloaders
path = os.path.join(os.getcwd(), "data/")

dataset, train_indices, test_indices= get_mimic_dataset(path, train_fraction = 0.5, test_fraction = 0.2)

TypeError: get_mimic_dataset() got an unexpected keyword argument 'train_fraction'

In [ ]:

train_loader, test_loader= get_mimic_dataloaders(dataset, train_indices, test_indices, batch_size=128)

In [ ]:
n_features = dataset.x.shape[1]
print(f"Number of features: {n_features}")

# Train the model
if not os.path.exists("target"):
    os.makedirs("target")
model = MimicLR(n_features)
train_acc, train_loss, test_acc, test_loss = create_trained_model_and_metadata(model, 
                                                                               train_loader, 
                                                                               test_loader, 
                                                                               lr = 0.0001,
                                                                                weight_decay = 5.392,
                                                                               epochs=2)




In [ ]:
from mimic_handler import MimicInputHandler

from leakpro import LeakPro

# Read the config file
config_path = "audit.yaml"

# Prepare leakpro object
leakpro = LeakPro(MimicInputHandler, config_path)

# Run the audit 
leakpro.run_audit()